Poniższy notatnik ma na celu zaprezentować działanie protokołu Diffiego-Hellmana na krzywych eliptycznych za pomocą biblioteki `cryptography`.

In [ ]:
!pip install cryptography

Importujemy potrzebne moduły z biblioteki `cryptography`.

- `ec`  to moduł, który zawiera implementacje logiki dla krzywych w standardowej postaci Weierstrassa, takich jak **`secp256r1`** (NIST P-256).
- `x25519` to moduł dla krzywej **Curve25519**.
- `serialization` to narzędzia pomocnicze, które pozwolą nam zamienić obiekty kluczy (będące punktami na krzywej) na format tekstowy, który można by "wysłać" przez sieć.

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec, x25519
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization


# `secp256r1`

In [ ]:
curve_secp256r1=ec.SECP256R1()
print(f"Poziom bezpieczeństwa (rozmiar klucza w bitach): {curve_secp256r1.key_size}")

Poziom bezpieczeństwa (rozmiar klucza w bitach): 256


Generujemy tajne klucze $a$ oraz $b$ dla Alicji i Boba. Następnie klucze publiczne czyli odpowiednio $aG$=$G_A$ oraz $bG$=$G_B$.

In [ ]:
private_key_alice=ec.generate_private_key(curve_secp256r1)

In [ ]:
public_key_alice = private_key_alice.public_key()

In [ ]:
private_key_bob=ec.generate_private_key(curve_secp256r1)
public_key_bob = private_key_bob.public_key()

W realnym scenariuszu klucze prywatne `a` i `b` nigdy nie opuszczają komputerów Alicji i Boba. Wymieniają oni **wyłącznie** swoje klucze publiczne (`G_A` i `G_B`) przez niezabezpieczony kanał. Zobaczmy jak wygląda obiekt wysłanny przez Alicję do Boba.



In [ ]:
public_key_bytes_alice = public_key_alice.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

print("--- Klucz publiczny Alicji (co widzi Bob) ---")
print(public_key_bytes_alice.decode('utf-8'))

--- Klucz publiczny Alicji (co widzi Bob) ---
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEwqWPXAZWPwUm1WirYTlM+8UGSz6Q
sK6EJ8yqlN4uhEbJ6GM7NDEWT8V6cDtUJKlEh+Fx7fECR2TMQc+1+UXHLA==
-----END PUBLIC KEY-----



Najpierw "serializujemy" klucz publiczny Alicji do formatu PEM – jest to standardowy format tekstowy, który Alicja wysłałaby Bobowi.

- `public_bytes()` to metoda wywoływana na obiekcie klucza. zapisuje ona obiekt w formie bajtów.

- `encoding=serialization.Encoding.PEM` to argument, który określa format kodowania dla wyjściowych bajtów.

- **PEM (Privacy-Enhanced Mail)** to standard, który bierze surowe binarne dane klucza, koduje je w Base64 (aby były czystym tekstem) i opakowuje w czytelne nagłówki. Jest to format tekstowy, więc jest bezpieczny do przesyłania np. w e-mailu lub w ciele żądania HTTP.

- `format=serialization.PublicFormat.SubjectPublicKeyInfo` Ten argument określa strukturę danych klucza. Jest to kluczowe dla interoperacyjności. **SubjectPublicKeyInfo** to standard X.509, który definiuje, jak opisać klucz publiczny. Zamiast wysyłać tylko surowe współrzędne (x, y) punktu G_A, ten format zawiera kompletny informację, że jest to klucz oparty na krzywej eliptycznej (ECC), identyfikator standaryzowanej krzywej (secp256r1), której użyliśmy. Dopiero na końcu właściwe współrzędne punktu publicznego G_A.

Dzięki temu, gdy Bob otrzyma te dane, jego oprogramowanie (nawet jeśli jest napisane w innym języku) odczyta identyfikator, załaduje te same parametry krzywej (p, a, b, G) i będzie wiedziało, jak poprawnie zinterpretować punkt G_A.

Ponieważ jest to symulacja, "zaglądnijmy Alicji przez ramię" i zobaczmy jej tajną wartość $a$ oraz surowe współrzędne $(x, y)$ jej klucza publicznego $G_A$. Robimy to za pomocą metody `private_numbers()`.


In [ ]:
wartosc_prywatna_a = private_key_alice.private_numbers().private_value

In [ ]:
wartosc_prywatna_a

100671325385697607855954757090459476668233595171984228303200380578994844132074

In [ ]:
liczby_publiczne_alice = public_key_alice.public_numbers()
wspolrzedna_x = liczby_publiczne_alice.x
wspolrzedna_y = liczby_publiczne_alice.y

In [ ]:
print(f"Klucz prywatny 'a' (TAJNE):\n{wartosc_prywatna_a}\n")
print(f"Klucz publiczny 'G_A' (PUBLICZNE):")
print(f"  Współrzędna x: {wspolrzedna_x}")
print(f"  Współrzędna y: {wspolrzedna_y}")

Klucz prywatny 'a' (TAJNE):
100671325385697607855954757090459476668233595171984228303200380578994844132074

Klucz publiczny 'G_A' (PUBLICZNE):
  Współrzędna x: 88041211821552150554714839284612843118236375760102141170503961195500339627078
  Współrzędna y: 91325475953286747306166507634418504919134300562629229283590794139717320689452


Przechodzimy do sedna ECDH czyli obliczenia wspólnego klucza Alicji i Boba, a następnie sprawdzenia czy są one takie same.

- `private_key_alice.exchange()` to metoda wywoływana na obiekcie klucza prywatnego Alicji. Pobiera ona tajną liczbę a (przechowywaną w private_key_alice) i wykonuje kluczową operację matematyczną: mnożenie punktu przez skalar. W tym przypadku oblicza S=a⋅G
B


- `ec.ECDH()` to argument, który instruuje metodę exchange, jakiego protokołu ma użyć do sfinalizowania klucza. Zamiast zwracać surowy punkt S = (ab)G, ten obiekt nakazuje wykonanie standardowej procedury, czyli wzięcie współrzędnej x z obliczonego punktu S, przekazanie tej współrzędnej przez Funkcję Wyprowadzania Klucza (KDF), która używa funkcji skrótu (np. SHA-256).

- `public_key_bob` to drugi argument, dostarczający dane wejściowe dla metody exchange.

In [ ]:
shared_key_alice = private_key_alice.exchange(ec.ECDH(), public_key_bob)

In [ ]:
shared_key_bob = private_key_bob.exchange(ec.ECDH(), public_key_alice)

In [ ]:
print(f"Sekret obliczony przez Alicję: {shared_key_alice.hex()}")
print(f"Sekret obliczony przez Boba:   {shared_key_bob.hex()}")

Sekret obliczony przez Alicję: 0d73bc34e5be39612c424e301ba864067fd8010e1a4737f662c077e78d79f477
Sekret obliczony przez Boba:   0d73bc34e5be39612c424e301ba864067fd8010e1a4737f662c077e78d79f477


In [ ]:
shared_key_alice == shared_key_bob

True

#`X25519`

Generujemy klucze prywatne oraz publiczne dla Alicji oraz Boba na podobnej zasadzie co powyżej.

In [ ]:
private_key_alice_x25519 = x25519.X25519PrivateKey.generate()

In [ ]:
public_key_alice_x25519 = private_key_alice_x25519.public_key()

In [ ]:
private_key_bob_x25519 = x25519.X25519PrivateKey.generate()
public_key_bob_x25519 = private_key_bob_x25519.public_key()

In [ ]:
shared_key_alice_x25519 = private_key_alice_x25519.exchange(public_key_bob_x25519)
shared_key_bob_x25519 = private_key_bob_x25519.exchange(public_key_alice_x25519)

In [ ]:
print(f"Sekret obliczony przez Alicję (X25519): {shared_key_alice_x25519.hex()}")
print(f"Sekret obliczony przez Boba (X25519):   {shared_key_bob_x25519.hex()}")

Sekret obliczony przez Alicję (X25519): 0770806eab7318bc72d29b56ef0f6c8a5034b081cdf9e93125d5508fc4a48574
Sekret obliczony przez Boba (X25519):   0770806eab7318bc72d29b56ef0f6c8a5034b081cdf9e93125d5508fc4a48574


In [ ]:
shared_key_alice_x25519 == shared_key_bob_x25519

True

W przypadku krzywej X25519, która jest krzywą Montgomery'ego, nie ma łatwo dostępnych "surowych" współrzędnych (x, y) punktów publicznych, tak jak w przypadku krzywych Weierstrassa (np. secp256r1). Jest to związane z optymalizacjami i sposobem, w jaki operacje są wykonywane na tej krzywej, co zwiększa bezpieczeństwo i wydajność.

Biblioteka `cryptography` abstrahuje te szczegóły i udostępnia metody do generowania kluczy i przeprowadzania wymiany kluczy (`exchange()`) w sposób bezpieczny i zgodny ze standardami dla X25519. Nie ma bezpośredniej metody, aby "podejrzeć" wewnętrzną wartość prywatną ani surowe współrzędne punktu publicznego w łatwo interpretowalnej formie liczbowej, ponieważ nie są one reprezentowane w ten sam sposób co w secp256r1.

